# Moving Chain without coupler
In this notebook, we implemented moving a ferromagnetic domain composed of three spins moving across a paramagnetic domain composed of three spins. The coupler is omitted. The dynamics remains adiabatic under sufficient slow update of the Zeeman field.

In [1]:
import numpy as np
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from ising_kitaev import initialize_chain, run_adiabatic_zeeman_change, rotate_to_measurement_basis, add_measurement
from ising_kitaev import move_chain

## Input parameters

In [2]:
zeeman_ferro = 0.01  # value of on-site magnetic field for ferromagnetic domain
zeeman_para = 10  # value of on-site magnetic field for paramagnetic domain
initial_config = np.array([zeeman_ferro, zeeman_ferro, zeeman_ferro, zeeman_para, zeeman_para, zeeman_para])
final_config = np.array([zeeman_para, zeeman_para, zeeman_para, zeeman_ferro, zeeman_ferro, zeeman_ferro])
qreg = QuantumRegister(6)
creg = ClassicalRegister(3)
qcirc = QuantumCircuit(qreg, creg)

## Preparing the initial state

In [3]:
initialize_chain(qcirc, qreg, initial_config, 'logical_zero')
qcirc.draw()
move_chain(qcirc, qreg, initial_config, final_config, 0, 0.25, 0.25, 2, 10, method = "both")
qcirc.depth()

8613

## Measurement

In [4]:
rotate_to_measurement_basis(qcirc, qreg, [3, 4, 5])
add_measurement(qcirc, qreg, creg, [3, 4, 5])

In [5]:
from qiskit import Aer, execute
backend = Aer.get_backend('qasm_simulator')
job = execute(qcirc, backend, shots=2000)
job.status()
result = job.result()
result.get_counts()

{'111': 5, '001': 30, '010': 6, '100': 4, '000': 1955}

We observe that we transfer with high fidelity the state accross the chain.